# 图像卷积

## 1. 互相关运算（cross-correlation）

在卷积层中，输入张量和核张量通过互相关运算产生输出张量

$$(n_h - k_h + 1) \times (n_w - k_w + 1)$$

我们可以通过在图像边界周围填充零来保证有足够的空间移动卷积核，从而保持输出大小不变。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


def corr2d(X, K):
    '''计算二维互相关运算'''
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j:j + w] * K).sum()
    return Y  

In [7]:
'''验证函数'''
X = torch.arange(9).reshape(3, 3)
Y = torch.arange(4).reshape(2, 2)
corr2d(X, Y)

tensor([[19., 25.],
        [37., 43.]])

## 2. 卷积层

对互相关运算的结果，添加标量偏置之后产生输出。

因此卷积层中的两个被训练的参数是卷积核权重和标量偏置。在训练前也需要随机初始化卷积核权重

In [8]:
class Conv2D(nn.Module):
    def __init__(self, kernal_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernal_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

## 3. 图像中目标的边缘检测

以下是卷积层的一个简单应用，通过找到像素变化的位置，检测图形中不同颜色的边缘

In [9]:
'''构建一个6*8像素的黑白图像，中间四列为黑色(0)，其余像素为白色(1)'''
X = torch.ones((6, 8))
X[:, 2: 6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [13]:
'''接下来构建一个高度为1，宽度为2的卷积核K，进行互相关运算时，水平相邻的两元素相同，输出为0，否则为非0'''
K = torch.tensor([1.0, -1.0], dtype=torch.float32).reshape(1, 2)

In [16]:
'''1表示白色到黑色的边缘
-1表示从黑色到白色的边缘
其他情况的输出为0
'''
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [17]:
'''将图像转置，在进行互相关运算
之前检测到的垂直边缘消失了
这个卷积核K值可以检测到  垂直  边缘，无法检测  水平  边缘
'''
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 4. 学习卷积核

对于更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以学习由X生成Y的卷积核？

我们先构造一个卷积层，然后将其卷积核初始化为随机张量。截下来在每次迭代中，比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单期间，再次使用内置的二维卷积层，并忽略偏置

In [33]:
# 构造一个二维卷积层，具有一个输出通道核形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size = (1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度）
# 其中批量大小和通道数都维1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 6.329
epoch 4, loss 1.388
epoch 6, loss 0.366
epoch 8, loss 0.116
epoch 10, loss 0.042


In [34]:
'''在10次迭代之后，误差已经降到足够低。
查看我们所学的卷积核的权重张量
'''
conv2d.weight.data

tensor([[[[ 1.0085, -0.9683]]]])